In [1]:
#| default_exp api

In [2]:
#|hide
from fastdownload import FastDownload

In [3]:
#|export
import pandas as pd
import logging
from fastcore.all import *
from hits_recsys.collab import *
from pathlib import Path
from typing import Literal, Optional
from fastai.collab import CollabDataLoaders, to_device

In [ ]:
#|export
class CollabModelService:
    def __init__(self, df, model) -> None:
        pass
    def predict(self, movie_names, ratings):
        self.movies.encode(movie_names)

In [4]:
#|export
@call_parse
def cli(optype, # operation to peroform, one of 'train', 'eval' or 'pred'
        r_path, # path to dataset with ratings
        m_path,  # path to dataset with movie titles
        model: Optional[Path]=None, # path to model if not train
        out: Path = './out.pkl'):  # path for output model, by default will save to './out.pkl'
    assert optype in ['train','eval','pred'], 'incorrect operation type'
    
    logging.info(f"loading datasets from {r_path} and {m_path}")
    df = read_movielens(r_path,m_path)
    logging.info(f"datasets loaded")
    
    dls = dataloaders(df)
    m = CollabUserBased()
    
    logging.info(f"start operation: {optype}")
    if optype=='train':
        train(m, dls, out)
        logging.info(f"model trained and saved to {out}")
   
    assert model is not None, 'model path must be not None'
    m.load(model)
    if optype=='eval':
        dls = dataloaders(df, bs=1024)
        loss = eval(m,dls.train)
        logging.info(f"loss = {loss.item()}")
    if optype=='pred':
        tdl = dls.test_dl(df[['userId','title']],bs=1024)
        preds = pred(m,tdl).tolist()
        res = pd.Series(preds)
        res.to_csv(out)
        logging.info(f"preds are saved to {out}")

In [5]:
url = 'https://raw.githubusercontent.com/MenshikovDmitry/TSU_AI_Course/main/module_1.%20Recommender%2BDevOps/dataset/'
files = ('ratings_train.dat ratings_test.dat movies.dat users.dat').split()
d = FastDownload()

In [6]:
paths = L(d.download(url+f) for f in files); paths

(#4) [Path('/home/slakter/.fastdownload/archive/ratings_train.dat'),Path('/home/slakter/.fastdownload/archive/ratings_test.dat'),Path('/home/slakter/.fastdownload/archive/movies.dat'),Path('/home/slakter/.fastdownload/archive/users.dat')]

In [7]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [8]:
cli('pred', paths[1], paths[2], './out.pkl', './out.csv')

INFO:root:loading datasets from /home/slakter/.fastdownload/archive/ratings_test.dat and /home/slakter/.fastdownload/archive/movies.dat
INFO:root:datasets loaded
INFO:root:start operation: pred


INFO:root:preds are saved to ./out.csv


## Web server

In [12]:
from fastapi import FastAPI
from importlib import metadata

In [9]:
app = FastAPI()

@app.get("/api/predict")
async def predict(movie_names: list, ratings: list):
    return {"message": "Hello World"}

@app.get("/api/reload")
async def reload():
    return {"message": "Hello World"}

@app.get("/api/similar")
async def similar():
    return {"message": "Hello World"}


In [ ]:
@app.get("/api/info")
async def info():
    return dict(metadata.metadata('hits-recsys'))

@app.get("/api/log")
async def log():
    return 

In [ ]:
import nest_asyncio
import uvicorn

if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app,port=8080)